In [2]:
import numpy as np
import pandas as pd
import os
from docx import Document
from docx.shared import Pt
from scipy.stats import gmean


In [3]:
# Constants
RI = 1.51

def read_pairwise_comparison_data(file_path):
    data = pd.read_csv(file_path, index_col=0)
    matrices = {}
    for col in data.columns:
        matrices[col] = data[col].apply(lambda x: eval(x) if isinstance(x, str) else x).values.reshape(-1, len(data))
    return matrices

In [4]:
def calculate_weights(matrix):
    eigvals, eigvecs = np.linalg.eig(matrix)
    max_eigval = np.max(eigvals)
    max_eigvec = eigvecs[:, np.argmax(eigvals)].real
    weights = max_eigvec / np.sum(max_eigvec)
    return weights, max_eigval


In [5]:
def calculate_consistency(matrix, max_eigval):
    n = len(matrix)
    CI = (max_eigval - n) / (n - 1)
    CR = CI / RI
    return CI, CR

In [6]:
def save_matrices_to_word(doc, matrices, title):
    for idx, (key, matrix) in enumerate(matrices.items(), 1):
        doc.add_heading(f'{title} for Clinical Engineer {key}', level=2)
        table = doc.add_table(rows=matrix.shape[0] + 1, cols=matrix.shape[1] + 1)
        table.style = 'Table Grid'
        
        for i in range(matrix.shape[0]):
            table.cell(i + 1, 0).text = f'Criterion {i + 1}'
            table.cell(0, i + 1).text = f'Criterion {i + 1}'
            for j in range(matrix.shape[1]):
                cell = table.cell(i + 1, j + 1)
                cell.text = f'{matrix[i, j]:.4f}'
                for paragraph in cell.paragraphs:
                    for run in paragraph.runs:
                        run.font.size = Pt(10)
        
        weights, max_eigval = calculate_weights(matrix)
        CI, CR = calculate_consistency(matrix, max_eigval)
        
        # Add calculated values to the document
        doc.add_paragraph(f'Weights: {weights}')
        doc.add_paragraph(f'Max Eigenvalue: {max_eigval:.4f}')
        doc.add_paragraph(f'Consistency Index (CI): {CI:.4f}')
        doc.add_paragraph(f'Consistency Ratio (CR): {CR:.4f}')

In [7]:
def aggregate_matrices(matrices):
    valid_matrices = [matrix for matrix in matrices.values() if calculate_consistency(matrix, calculate_weights(matrix)[1])[1] <= 0.1]
    if not valid_matrices:
        raise ValueError("No valid matrices with CR <= 0.1")
    aggregate_matrix = gmean(np.array(valid_matrices), axis=0)
    return aggregate_matrix

In [8]:
def process_criteria(file_path, doc, title):
    matrices = read_pairwise_comparison_data(file_path)
    save_matrices_to_word(doc, matrices, f'{title} Matrices')
    
    aggregate_matrix = aggregate_matrices(matrices)
    weights, max_eigval = calculate_weights(aggregate_matrix)
    CI, CR = calculate_consistency(aggregate_matrix, max_eigval)
    
    doc.add_heading(f'Aggregate {title} Matrix', level=2)
    table = doc.add_table(rows=aggregate_matrix.shape[0] + 1, cols=aggregate_matrix.shape[1] + 1)
    table.style = 'Table Grid'
    
    for i in range(aggregate_matrix.shape[0]):
        table.cell(i + 1, 0).text = f'Criterion {i + 1}'
        table.cell(0, i + 1).text = f'Criterion {i + 1}'
        for j in range(aggregate_matrix.shape[1]):
            cell = table.cell(i + 1, j + 1)
            cell.text = f'{aggregate_matrix[i, j]:.4f}'
            for paragraph in cell.paragraphs:
                for run in paragraph.runs:
                    run.font.size = Pt(10)
    
    doc.add_paragraph(f'Weights: {weights}')
    doc.add_paragraph(f'Max Eigenvalue: {max_eigval:.4f}')
    doc.add_paragraph(f'Consistency Index (CI): {CI:.4f}')
    doc.add_paragraph(f'Consistency Ratio (CR): {CR:.4f}')

    return weights

In [9]:
# Paths
criteria_file = os.path.expanduser('./data/criteria.csv')
subcriteria_files = {
    "Performance": os.path.expanduser('./data/performance.csv'),
    "Physical Construction and Quality": os.path.expanduser('./data/physical_construction.csv'),
    "Compliance with Regulations and Standards": os.path.expanduser('./data/compliance.csv'),
    "Reliability and Robustness": os.path.expanduser('./data/reliability.csv'),
    "Ease of Maintenance": os.path.expanduser('./data/ease_of_maintenance.csv'),
    "Safety": os.path.expanduser('./data/safety.csv'),
    "Compatibility with existing Systems and Devices": os.path.expanduser('./data/compatibility.csv'),
    "Ease of Use": os.path.expanduser('./data/ease_of_use.csv'),
    "Total Cost of Ownership": os.path.expanduser('./data/cost_of_ownership.csv'),
    "Environmental and Utilities Requirements": os.path.expanduser('./data/environmental_requirements.csv'),
    "Supplier": os.path.expanduser('./data/supplier.csv'),
}
print (subcriteria_files)
doc = Document()
doc.add_heading('AHP-TOPSIS Model for Health Technology Assessment of Medical Equipment', 0)

# # Process main criteria
main_criteria_weights = process_criteria(criteria_file, doc, "Main Criteria")

# Process subcriteria
subcriteria_weights = {}
for criterion, file_path in subcriteria_files.items():
    subcriteria_weights[criterion] = process_criteria(file_path, doc, f'Subcriteria for {criterion}')

# Save document
doc.save('AHP_TOPSIS_Results.docx')

{'Performance': './data/performance.csv', 'Physical Construction and Quality': './data/physical_construction.csv', 'Compliance with Regulations and Standards': './data/compliance.csv', 'Reliability and Robustness': './data/reliability.csv', 'Ease of Maintenance': './data/ease_of_maintenance.csv', 'Safety': './data/safety.csv', 'Compatibility with existing Systems and Devices': './data/compatibility.csv', 'Ease of Use': './data/ease_of_use.csv', 'Total Cost of Ownership': './data/cost_of_ownership.csv', 'Environmental and Utilities Requirements': './data/environmental_requirements.csv', 'Supplier': './data/supplier.csv'}


LinAlgError: Last 2 dimensions of the array must be square